In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import os
import pandas as pd

In [2]:
spark = (
    SparkSession.builder.
    appName("Working with tabular data").
    getOrCreate())

In [3]:
PATH = r"C:\Users\oluwa\Documents\broadcast_logs"

In [5]:
logs = spark.read.csv(os.path.join(PATH, 'BroadcastLogs_2018_Q3_M8.csv'),
                      header=True,
                      sep="|",
                      inferSchema=True,
                      timestampFormat="yyyy-MM-dd"
                     )

In [6]:
logs.printSchema()

root
 |-- BroadcastLogID: integer (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: timestamp (nullable = true)
 |-- SequenceNO: integer (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: timestamp (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string 

In [11]:
logs.columns

['BroadcastLogID',
 'LogServiceID',
 'LogDate',
 'SequenceNO',
 'AudienceTargetAgeID',
 'AudienceTargetEthnicID',
 'CategoryID',
 'ClosedCaptionID',
 'CountryOfOriginID',
 'DubDramaCreditID',
 'EthnicProgramID',
 'ProductionSourceID',
 'ProgramClassID',
 'FilmClassificationID',
 'ExhibitionID',
 'Duration',
 'EndTime',
 'LogEntryDate',
 'ProductionNO',
 'ProgramTitle',
 'StartTime',
 'Subtitle',
 'NetworkAffiliationID',
 'SpecialAttentionID',
 'BroadcastOriginPointID',
 'CompositionID',
 'Producer1',
 'Producer2',
 'Language1',
 'Language2']

In [14]:
logs.select('BroadcastLogID',
 'LogServiceID',
 'LogDate').show(5, truncate=False)

+--------------+------------+-------------------+
|BroadcastLogID|LogServiceID|LogDate            |
+--------------+------------+-------------------+
|1196192316    |3157        |2018-08-01 00:00:00|
|1196192317    |3157        |2018-08-01 00:00:00|
|1196192318    |3157        |2018-08-01 00:00:00|
|1196192319    |3157        |2018-08-01 00:00:00|
|1196192320    |3157        |2018-08-01 00:00:00|
+--------------+------------+-------------------+
only showing top 5 rows



In [23]:
#exclude 'BroadcastLogID' and 'SequenceNO' columns

logs = logs.select(*[x for x in logs.columns if x not in ['BroadcastLogID', 'SequenceNO', 'LogServiceID']])

In [24]:
logs.columns

['LogDate',
 'AudienceTargetAgeID',
 'AudienceTargetEthnicID',
 'CategoryID',
 'ClosedCaptionID',
 'CountryOfOriginID',
 'DubDramaCreditID',
 'EthnicProgramID',
 'ProductionSourceID',
 'ProgramClassID',
 'FilmClassificationID',
 'ExhibitionID',
 'Duration',
 'EndTime',
 'LogEntryDate',
 'ProductionNO',
 'ProgramTitle',
 'StartTime',
 'Subtitle',
 'NetworkAffiliationID',
 'SpecialAttentionID',
 'BroadcastOriginPointID',
 'CompositionID',
 'Producer1',
 'Producer2',
 'Language1',
 'Language2']

In [29]:
logs.select(F.col('LogDate')).show(5, False)

+-------------------+
|LogDate            |
+-------------------+
|2018-08-01 00:00:00|
|2018-08-01 00:00:00|
|2018-08-01 00:00:00|
|2018-08-01 00:00:00|
|2018-08-01 00:00:00|
+-------------------+
only showing top 5 rows



In [30]:
logs.select(F.substring(F.col('LogDate'), 1, 4)).distinct().show(5)

+------------------------+
|substring(LogDate, 1, 4)|
+------------------------+
|                    2018|
+------------------------+



In [31]:
logs.select(
    F.col('Duration').alias("full_duration")).show(5, False)

+----------------+
|full_duration   |
+----------------+
|02:00:00.0000000|
|00:00:30.0000000|
|00:00:15.0000000|
|00:00:15.0000000|
|00:00:15.0000000|
+----------------+
only showing top 5 rows



In [54]:
logs.select(
    F.col('Duration').alias('full'),
    F.col('Duration').substr(1,2).cast('int').alias('hour'),
    F.col('Duration').substr(4,2).cast('int').alias('min'),
    F.col('Duration').substr(7,2).cast('int').alias('sec'),
    (F.col('Duration').substr(1,2).cast('int') * 60 * 60 +
    F.col('Duration').substr(4,2).cast('int') * 60 +
    F.col('Duration').substr(7,2).cast('int')).alias('duration_secs')
).distinct().show(5, False)

+----------------+----+---+---+-------------+
|full            |hour|min|sec|duration_secs|
+----------------+----+---+---+-------------+
|00:59:52.0000000|0   |59 |52 |3592         |
|00:16:31.0000000|0   |16 |31 |991          |
|00:13:30.0000000|0   |13 |30 |810          |
|00:48:17.0000000|0   |48 |17 |2897         |
|00:00:19.0000000|0   |0  |19 |19           |
+----------------+----+---+---+-------------+
only showing top 5 rows



In [64]:
logs = logs.withColumn('duration_secs',
F.col('Duration').substr(1,2).cast('int') * 60 * 60 +
F.col('Duration').substr(4,2).cast('int') * 60 +
F.col('Duration').substr(7,2).cast('int'))

In [65]:
logs.printSchema()

root
 |-- LogDate: timestamp (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: timestamp (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable = true)
 |-- NetworkAffiliationID: integer (nullable = true)
 |-- SpecialAttentionID: integer (nullable = true)
 |-- Broadcas